# Script pour rejouer un certain modèle de Q-learning
Source: https://medium.com/@www.seymour/training-an-ai-to-play-a-game-using-deep-reinforcement-learning-b63534cfdecd

Dans ce script, il n'y a qu'à changer le numéro et chemin du modèle pour ensuite voir le rendu de l'agent

### Imports

In [110]:
import numpy as np
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src import ql, game, record, env

import importlib
importlib.reload(game)
importlib.reload(ql)
importlib.reload(env)

import pygame

### Modèle à charger

In [111]:
model_dir = os.path.join("..", "simulations", "2024-04-18 1757", "models")
video_dir = "video"
model_name = "q_table_final"
record_video = False
WINDOW_WIDTH = 360
WINDOW_HEIGHT = 360

### Chargement des éléments de jeu

In [112]:

#game = "CliffWalking-v0"
#game = "Catcher"
game = "FrozenLake-v1"
#game = "Taxi-v3"
if game == "Catcher":
    env = env.env(game)
else:
    env = env.env(game, render_mode="human")
agent = ql.QLearning(
    state_size=env.state_space,
    action_size=env.action_space
)
agent.load(os.path.join(model_dir, f"{model_name}.npy"))

### Script de replay

In [113]:
state = env.reset()

pygame.init()
if game == "Catcher":
    screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
else:
    screen = None
clock = pygame.time.Clock()
running = True
score = 0

recorder = None
if record_video:
    recorder = record.ScreenRecorder(WINDOW_WIDTH, WINDOW_HEIGHT, env.env.FPS, f"{video_dir}_{model_name}.avi")

while running:
    pygame.display.set_caption(f"Score: {score}")

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    action = agent.act(state, 0)
    state, reward, done, score = env.step(action)

    env.render(screen)
    pygame.display.flip()
    clock.tick(15)

    if record_video:
        recorder.capture_frame(screen)
    if game != "Catcher" and done:
        break

pygame.quit()
if record_video:
    recorder.end_recording()